In [12]:
import pandas as pd
import os
from konlpy.tag import Okt
import requests
import re

# Pig 데이터가 저장된 경로
base_path = '../pig/result/'

# 리뷰 데이터 경로
review_data_path = '../data/university_cafe_reviews.csv'

# Pig 카테고리 리스트
pig_categories = [
    'high_rating_high_visit_high_review_cafes',
    'high_rating_high_visit_low_review_cafes',
    'high_rating_low_visit_high_review_cafes',
    'high_rating_low_visit_low_review_cafes',
    'low_rating_high_visit_high_review_cafes',
    'low_rating_high_visit_low_review_cafes',
    'low_rating_low_visit_high_review_cafes',
    'low_rating_low_visit_low_review_cafes'
]

# 형태소 분석기 초기화
okt = Okt()

# 불용어 정의
url = "https://raw.githubusercontent.com/stopwords-iso/stopwords-ko/master/stopwords-ko.txt"
response = requests.get(url)
stopwords = response.text.split("\n")  # 불용어 리스트 로드


# 이모티콘 제거 함수 정의
def remove_emojis(text):
    emoji_pattern = re.compile(
        "["
        u"\U0001F600-\U0001F64F"  # 감정 이모지
        u"\U0001F300-\U0001F5FF"  # 기호 및 물체 이모지
        u"\U0001F680-\U0001F6FF"  # 교통 및 기계 이모지
        u"\U0001F700-\U0001F77F"  # 기타 이모지
        u"\U0001F780-\U0001F7FF"  # 더 많은 기타 이모지
        u"\U0001F800-\U0001F8FF"  # 추가 범위
        u"\U0001F900-\U0001F9FF"  # 제스처, 스포츠 이모지
        u"\U0001FA00-\U0001FA6F"  # 오브젝트 및 심볼
        u"\U0001FA70-\U0001FAFF"  # 추가 심볼
        u"\u2600-\u26FF"  # 기타 기호
        u"\u2700-\u27BF"  # 문자 기호
        "]+",
        flags=re.UNICODE
    )
    return emoji_pattern.sub(r'', text)


# 리뷰 데이터 로드
review_data = pd.read_csv(review_data_path)

# 리뷰 데이터 컬럼 이름 정렬
review_data = review_data.rename(columns={
    "가게 평균 평점": "avgRating",
    "리뷰어 리뷰수": "totalReviewerCount",
    "가게 방문수": "totalVisitCount",
    "대학교 이름": "university",
    "가게 이름": "cafeName"
})

# 공백 제거
review_data.columns = review_data.columns.str.strip()
review_data["university"] = review_data["university"].str.strip()
review_data["cafeName"] = review_data["cafeName"].str.strip()

# 카테고리별 처리
for category in pig_categories:
    category_path = os.path.join(base_path, category)

    if not os.path.exists(category_path):
        print(f"경로가 존재하지 않습니다: {category_path}")
        continue

    pig_files = [os.path.join(category_path, file) for file in os.listdir(category_path) if file.startswith("part")]
    if not pig_files:
        print(f"경로에 'part' 파일이 없습니다: {category_path}")
        continue

    pig_data = pd.concat([pd.read_csv(file, header=None) for file in pig_files], ignore_index=True)

    # Pig 데이터 컬럼 이름 설정
    pig_data.columns = ["university", "cafeName", "avgRating", "totalReviewerCount", "totalVisitCount"]

    # 공백 제거
    pig_data.columns = pig_data.columns.str.strip()
    pig_data["university"] = pig_data["university"].str.strip()
    pig_data["cafeName"] = pig_data["cafeName"].str.strip()

    # Pig 데이터와 리뷰 데이터 Join
    merged_data = pig_data.merge(review_data, on=["university", "cafeName"], how="left")

    # 리뷰 텍스트 병합
    merged_data["reviewText"] = merged_data["리뷰 텍스트"].astype(str)
    merged_data["merged_text"] = merged_data.groupby(["university", "cafeName"])["reviewText"].transform(
        lambda x: " ".join(x))
    merged_data = merged_data.drop_duplicates(subset=["university", "cafeName"])

    # 텍스트 전처리: 이모티콘 제거, 특수문자 제거
    merged_data["cleaned_text"] = merged_data["merged_text"].apply(remove_emojis)
    merged_data["cleaned_text"] = merged_data["cleaned_text"].str.replace(r"[^\w\s]", "", regex=True)  # 특수문자 제거
    merged_data["cleaned_text"] = merged_data["cleaned_text"].str.replace(r"\d+", "", regex=True)  # 숫자 제거

    # 형태소 추출: 형용사(Adjective), 동사(Verb), 감탄사(Exclamation), 부사(Adverb)만 포함
    merged_data["processed_text"] = merged_data["cleaned_text"].apply(
        lambda x: " ".join([word for word, tag in okt.pos(x) if tag in ["Adjective", "Verb", "Exclamation", "Adverb"] and word not in stopwords])
    )

    # 병합된 텍스트 저장
    output_path = os.path.join(category_path, f"{category}_processed_text.txt")
    with open(output_path, "w", encoding="utf-8") as f:
        for text in merged_data["processed_text"].dropna():
            f.write(text + "\n")

    print(f"저장 완료: {output_path}")

저장 완료: ../pig/result/high_rating_high_visit_high_review_cafes/high_rating_high_visit_high_review_cafes_word_analysis.csv
저장 완료: ../pig/result/high_rating_high_visit_low_review_cafes/high_rating_high_visit_low_review_cafes_word_analysis.csv
저장 완료: ../pig/result/high_rating_low_visit_high_review_cafes/high_rating_low_visit_high_review_cafes_word_analysis.csv
저장 완료: ../pig/result/high_rating_low_visit_low_review_cafes/high_rating_low_visit_low_review_cafes_word_analysis.csv
저장 완료: ../pig/result/low_rating_high_visit_high_review_cafes/low_rating_high_visit_high_review_cafes_word_analysis.csv
저장 완료: ../pig/result/low_rating_high_visit_low_review_cafes/low_rating_high_visit_low_review_cafes_word_analysis.csv
저장 완료: ../pig/result/low_rating_low_visit_high_review_cafes/low_rating_low_visit_high_review_cafes_word_analysis.csv
저장 완료: ../pig/result/low_rating_low_visit_low_review_cafes/low_rating_low_visit_low_review_cafes_word_analysis.csv
